## Logistic regression model
### reference
- [demo](https://github.com/torch/demos/blob/master/logistic-regression/example-logistic-regression.lua)
- [coursera lecture](https://github.com/didw/lecture/blob/master/machine-learning/machine-learning-ex2/ex2/ex2.m)

In [32]:
require 'torch'
require 'nn'
require 'optim'
require 'csvigo'

In [33]:
loaded = csvigo.load{path='ex2data1.txt', mode='raw'}

In [34]:
data = torch.Tensor{ loaded }[1]:t()[{{1,2}}]:t()

<csv>	parsing file: ex2data1.txt	
<csv>	parsing done	


In [35]:
label = torch.Tensor{ loaded }[1]:t()[3] + 1

### Define the model

In [36]:
linLayer = nn.Linear(2,2)
softMaxLayer = nn.LogSoftMax()
model = nn.Sequential()
model:add(linLayer)
model:add(softMaxLayer)

### Define a loss function

In [37]:
criterion = nn.ClassNLLCriterion()

### Train the model (using SGD)

In [38]:
x, dl_dx = model:getParameters()

In [39]:
feval = function(x_new)
   if x ~= x_new then
      x:copy(x_new)
   end

   _nidx_ = (_nidx_ or 0) + 1
   if _nidx_ > (#data)[1] then _nidx_ = 1 end

   local inputs = data[_nidx_]
   local target = label[_nidx_]

   dl_dx:zero()

   local loss_x = criterion:forward(model:forward(inputs), target)
   model:backward(inputs, criterion:backward(model.output, target))

   return loss_x, dl_dx
end

In [40]:
sgd_params = {
   learningRate = 1e-3,
   learningRateDecay = 1e-4,
   weightDecay = 0,
   momentum = 0
}

In [41]:
epochs = 1e3

In [42]:
print('')
print('============================================================')
print('Training with SGD')
print('')

	
Training with SGD	
	


In [43]:
for i = 1,epochs do
   current_loss = 0
   for i = 1,(#data)[1] do
      _,fs = optim.sgd(feval,x,sgd_params)
      current_loss = current_loss + fs[1]
   end

   current_loss = current_loss / (#data)[1]
   if i % 100 == 1 then
        print('epoch = ' .. i .. 
         ' of ' .. epochs .. 
         ' current loss = ' .. current_loss)
    end
end

epoch = 1 of 1000 current loss = 4.6103405937477	


epoch = 101 of 1000 current loss = 1.7384717840075	


epoch = 201 of 1000 current loss = 1.1421569438176	

epoch = 301 of 1000 current loss = 0.87574057348067	


epoch = 401 of 1000 current loss = 0.74041319695385	


epoch = 501 of 1000 current loss = 0.66337342890834	


epoch = 601 of 1000 current loss = 0.61497904781015	


epoch = 701 of 1000 current loss = 0.58202394269726	


epoch = 801 of 1000 current loss = 0.55805004936938	


epoch = 901 of 1000 current loss = 0.53969694178098	


### Test the trained model

In [44]:
function maxIndex(a,b)
    if a>=b then return 1
    else return 2 end
end

In [45]:
function predictOut(a, b)
    local input = torch.Tensor(2)
    input[1] = a
    input[2] = b
    local logProbs = model:forward(input)
    local probs = torch.exp(logProbs)
    local prob1, prob2 = probs[1], probs[2]
    return maxIndex(prob1, prob2), prob1, prob2
end

In [46]:
corr = 0
for i = 1, (#data)[1] do
    local prediction = predictOut(data[i][1], data[i][2])
    
    -- print(prediction, label[i])
    if prediction == label[i] then
        corr = corr + 1
    end
end
print (corr / (#data)[1])

0.88	
